In [1]:
from inscopefilter3_eval import*
if __name__ == '__main__':
                #設定訓練參數和訓練模型存放路徑
    s=time()
    #batch_size = 3
    batch_size = 512
    #num_classes = 6
    #epochs = 2000
    epochs = 100
    seed=0
    #validation spilt
    spilt=0.1
    #for variance threshold
    #fp_dim=1e6
    fp_dim=16384
    recfp_dim=2048
    model_name = 'trained_model_inscope_'+str(seed)
    save_dir = os.path.join(os.getcwd(), 'saved_models')




    print('Loading data...')
    tem_simp = set()
    prods = []
    reacs = []
    labels = []
    '''
    with open('data/inscopedata.dat', 'r') as f:
        for l in tqdm(f, desc='inscopedata'):
            tem_simp.add(l.strip())

    with open('data/inscopedata2.dat', 'r') as f:
        for l in tqdm(f, desc='inscopedata'):
            tem_simp.add(l.strip()) 
    #print('check data:', tem_simp)

    with open('data/inscopedata4.dat', 'r') as f:
        for l in tqdm(f, desc='inscopedata'):
            tem_simp.add(l.strip()) 
        
    for item in tem_simp:
        prod,reac,label = item.split('\t')
        prods.append(prod)
        reacs.append(reac)
        labels.append(int(label))
    #print('check samples:', labels[1000000:1000010])
    print('total samples:', len(tem_simp))    
    # Shuffle
    xyz = list(zip(prods, reacs, labels))
    xyz.sort()
    random.seed(seed)
    random.shuffle(xyz)
    
    prods, reacs, labels = zip(*xyz)
    '''
    '''
    with open('data/inscopedatatest.dat', 'r') as f:
        for l in tqdm(f, desc='inscopedata'):
            tem_simp.add(l.strip()) 
        
    for item in tem_simp:
        prod,reac,label = item.split('\t')
        prods.append(prod)
        reacs.append(reac)
        labels.append(int(label))
        
    data_spilt= round(len(prods)*(1-spilt))
    x_train = prods[:data_spilt]
    x_test = prods[data_spilt:]
    y_train = reacs[:data_spilt]
    y_test = reacs[data_spilt:]
    z_train = labels[:data_spilt]
    z_test = labels[data_spilt:]
    '''
    #print('traindata:',x_train[:2],y_train[:2],z_train[:2])
    #print('testdata:',x_test[:2],y_test[:2],z_test[:2])
    
    with open('data/x_train.pickle', 'rb') as f:
        x_train = pickle.load(f)
    with open('data/x_test0.pickle', 'rb') as f:
        x_test = pickle.load(f)
    with open('data/y_train.pickle', 'rb') as f:
        y_train = pickle.load(f)
    with open('data/y_test0.pickle', 'rb') as f:
        y_test = pickle.load(f)
    with open('data/z_train.pickle', 'rb') as f:
        z_train = pickle.load(f)
    #with open('data/z_train0-smaM0.pickle', 'rb') as f:
    #    z_test = pickle.load(f) 
    with open('data/z_test0.pickle', 'rb') as f:
        z_test = pickle.load(f)   

    
    print('shuffle is over...')

    #build model
    visible = Input(shape=(fp_dim,))
    hidden = Lambda(lambda x: tf.math.log(x+1))(visible)
    hidden = Dense(1024, activation='elu')(hidden)
    hidden = Dropout(0.3)(hidden)

    # only for expansion rule policynet
    for _ in range(5):
        hidden = Highway()(hidden)
    #    hidden = Dropout(0.4)(hidden)
    #another branch
    #visible1 = Input(shape=(fp_dim,))
    visible2 = Input(shape=(recfp_dim,))
    #hidden1 = Lambda(fold)([visible, visible2])
    hidden1 = Dense(1024, activation='elu')(visible2)

    output = Lambda(cosine)([hidden, hidden1])
    #,output_shape=(1,)
    
    model = Model(inputs=[visible,visible2], outputs=output)
    # summarize layers
    print(model.summary())
    # plot graph
    #plot_model(model, to_file='expansionpolicynet_graph.png')
    # 初始化Adam optimizer
    opt = keras.optimizers.Adam(lr=0.0001)

    # 設定訓練方式，包含loss、optimizer..)
    loss1=losses.binary_crossentropy
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=[metrics.binary_accuracy, ACCR, auc1, TPR, FPR])
    #metrics.binary_accuracy, ACCR, auc2, auc1, TPR, FPR
    # early stop存放模型設置


    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    checkpoint = ModelCheckpoint(model_path, monitor='val_ACCR', save_best_only=True, verbose=1, mode='max')

    # early stop參數設定
    earlystop = EarlyStopping(monitor='val_ACCR', patience=6, verbose=1, mode='max')

    #continue training

    #K.clear_session()
    #gc.collect()
    del model  # 删掉存在的模型

    #返回一个编译好的模型
    #与删掉的模型相同
    model = load_model(model_path, custom_objects={'ACCR': ACCR,'auc2': auc2,'auc1': auc1,'TPR': TPR, 'FPR': FPR,'Highway': Highway,'fold': fold,'cosine': cosine, 'tf': tf, 'loss1': loss1})
    
    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=[metrics.binary_accuracy, ACCR, auc1, TPR, FPR, loss1])
    #metrics.binary_accuracy, ACCR, auc2, auc1, TPR, FPR
    # 開始訓練
    x_test1= x_test[:]
    y_test1= y_test[:]
    z_test1= z_test[:]
    
    training_generator = DataGenerator(X=x_train, y=y_train, z=z_train, batch_size=batch_size, shuffle=True, fp_dim=fp_dim, recfp_dim=recfp_dim)
    validation_gen = DataGenerator(X=x_test1, y=y_test1, z=z_test1, batch_size=batch_size, shuffle=True, fp_dim=fp_dim, recfp_dim=recfp_dim)    

    model_history = model.evaluate_generator( 
                    generator=validation_gen,
                    
                    
                    
                    verbose=0,
                    
                    workers=8, 
                    use_multiprocessing=1, 

#                    max_queue_size = 12, 

                    )
    print('Rollout took:', time() - s)

RDKit WARNING: [19:18:34] Enabling RDKit 2019.09.3 jupyter extensions
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Loading data...
shuffle is over...
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 16384)        0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 16384)        0           input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1024)         16778240    lambda_1[0][0]                   
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1024)         0           dense_1[0][0]      

In [2]:
model_history 


[0.6983139514923096,
 0.7602792382240295,
 0.8617534041404724,
 0.9276516437530518,
 0.9784990549087524,
 0.3716548979282379,
 0.6606940031051636]

In [3]:
model.metrics_names


['loss',
 'binary_accuracy',
 'ACCR',
 'auc1',
 'TPR',
 'FPR',
 'binary_crossentropy']

In [5]:
test1=model_history 
model_history 

[0.04164240509271622,
 0.9811465740203857,
 0.9436202049255371,
 0.9949966669082642,
 0.9922098517417908,
 0.07911659777164459,
 0.06194719299674034]

In [4]:
test2=model_history
#test3=[(a+b)/2 for a,b in zip(test1,test2)]
model_history 

[0.1387280523777008,
 0.9583728313446045,
 0.9748637080192566,
 0.979328989982605,
 0.9072006344795227,
 0.039099305868148804,
 0.12616582214832306]

In [6]:
z_test[:20]

(1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0)

In [9]:
model_history 

[0.1459132879972458,
 0.9658203125,
 0.974609375,
 0.9793407917022705,
 0.9671593904495239,
 0.8953201770782471,
 0.033569060266017914,
 0.14063453674316406]

In [5]:
print(model_history)

[0.08207101374864578, 0.9658203125, 0.9814453125, 0.9840490818023682, 0.6847290992736816, 0.005113622173666954]
